In [ ]:
import numpy as np
import random
from tabulate import tabulate
from my_parameters import ProblemParameters

class SimpleGA:
    def __init__(self, params):
        self.params = params
        self.prob_matr, self.y_matr= params.compute_d_prob()
        self.min_expec=self.min_expectation(self.prob_matr,self.y_matr)
        self.pop_size = 50  # 明确指定种群大小
        
        # 生成30个可行解的种群
        print(f"正在初始化{self.pop_size}个可行解...")
        self.population =self._generate_population()

        print("初始化完成\n")
    
    
    def min_expectation(self, prob_matr, y_matr):
        min_exp = {}
        max_alloc = max(np.max(self.params.S), np.max(self.params.K))  # 最大分配量上限
        
        for j in range(self.params.d):
            for loc in range(self.params.r):
                # 动态步长（高需求区域更密集）
                step = max(1, int(self.params.lambdas[j, loc] / 10))
                x_vals = list(range(0, max_alloc + 1, step))
                
                # 向量化计算期望
                y_vals = [
                    np.sum(prob_matr[j, loc] * np.minimum(z, y_matr[j, loc]))
                    for z in x_vals
                ]
                
                min_exp[(j, loc)] = (x_vals, y_vals)
        return min_exp
    
    def _generate_population(self):
        # 批量生成多少个可行解（严格满足约束）
        population = []
        
        # 1. 基础分配：按地区容量50%分配（不超过供应量）
        base_alloc = np.zeros((self.params.d, self.params.r), dtype=int)
        remaining_S = self.params.S.copy()  # 各产品剩余供应量
        
        for r in range(self.params.r):
            # 该地区可分配总量 = 容量的一半
            total_alloc_r = int(self.params.K[r] * 0.5)
            
            # 按产品需求比例分配（确保不超过剩余供应量）
            for j in range(self.params.d):
                alloc = int(total_alloc_r * self.params.lambdas[j, r] / np.sum(self.params.lambdas[:, r]))
                base_alloc[j, r] = min(alloc, remaining_S[j])
                remaining_S[j] -= base_alloc[j, r]
        
        # 2. 为每个个体生成随机扰动
        for _ in range(self.pop_size):
            indiv = base_alloc.copy()
            remaining_S_indiv = self.params.S - np.sum(base_alloc, axis=1)  # 当前个体各产品剩余量
            remaining_capacity = self.params.K - np.sum(base_alloc, axis=0)  # 各地区剩余容量
            
            # 随机分配剩余量（严格约束版）
            for r in np.random.permutation(self.params.r):  # 随机选择地区顺序
                if remaining_capacity[r] <= 0:
                    continue
                    
                # 生成该地区的随机分配（确保不超剩余供应量）
                alloc_r = np.zeros(self.params.d, dtype=int)
                for j in np.random.permutation(self.params.d):  # 随机选择产品顺序
                    if remaining_S_indiv[j] <= 0 or remaining_capacity[r] <= 0:
                        continue
                        
                    max_alloc = min(remaining_S_indiv[j], remaining_capacity[r])
                    alloc_r[j] = random.randint(0, max_alloc)
                    remaining_S_indiv[j] -= alloc_r[j]
                    remaining_capacity[r] -= alloc_r[j]
                
                # 应用分配
                indiv[:, r] += alloc_r
            
            # 二次验证（重要！）
            if (not (all(np.sum(indiv, axis=1) <= self.params.S)) or 
                    not (all(np.sum(indiv, axis=0) <= self.params.K))):
                # 如果违反约束，重新生成该个体
                continue  
            population.append(indiv)
            if len(population) >= self.pop_size:
                break
        
        return population
            
            


    def run(self, max_gens=30):
        """带精英保留的遗传算法主循环，保持表格输出格式"""
        print("=== 遗传算法开始 ===")
        header = ["代数", "最佳值", "平均值", "最差值", "最佳分配示例"] 
        table_data = []
        
        best_global = {
            'fitness': -np.inf,
            'solution': None
        }  # 记录全局最优解
    
        for gen in range(max_gens):
            # 1. 评估当前种群
            fitness = [self._evaluate(indiv) for indiv in self.population]
            best_idx = np.argmax(fitness)
            avg_fit = np.mean(fitness)
            worst_idx = np.argmin(fitness)
            
            # 记录日志
            table_data.append([
                gen,
                f"{fitness[best_idx]:.1f}",
                f"{avg_fit:.1f}",
                f"{fitness[worst_idx]:.1f}",
                f"产品0: {self.population[best_idx][0]}"
            ])
            
            # 2. 精英选择 (保留前5%的最优个体)
            elite_size = max(1, int(self.pop_size * 0.05))  # 至少保留1个精英
            elites = [x[1] for x in sorted(
                zip(fitness, self.population), 
                key=lambda x: -x[0]
            )[:elite_size]]
            
            # 3. 选择父母 (保留前50%的个体作为候选父母)
            parents = [x[1] for x in sorted(
                zip(fitness, self.population), 
                key=lambda x: -x[0]
            )[:self.pop_size//2]]
            
            # 4. 生成新一代
            new_pop = elites.copy()  # 首先注入精英
            
            while len(new_pop) < self.pop_size:
                # 4.1 80%概率进行交叉，20%概率直接复制父母
                if random.random() < 0.8 and len(parents) >= 2:
                    p1, p2 = random.sample(parents, 2)
                    child = self._crossover(p1, p2)
                else:
                    child = random.choice(parents).copy()
                
                # 4.2 对非精英个体进行变异
                if len(new_pop) > elite_size:  # 精英不参与变异
                    child = self._mutate(child)
                
                new_pop.append(child)
            
            # 5. 每3代注入一个全新随机个体（维持多样性）
            if gen % 3 == 0 and len(new_pop) >= 1:
                new_pop[-1] = self._generate_random_individual()
            
            self.population = new_pop
            
            # # 6. 打印当前精英信息
            # print(f"Gen {gen}: 精英适应度 = {fitness[best_idx]:.1f}")
        
        # 最终结果输出
        print(tabulate(table_data, headers=header, tablefmt="grid"))
        
        # 返回历史最佳解（可能不在最后一代）
        best_idx = np.argmax([self._evaluate(indiv) for indiv in self.population])
        return self.population[best_idx]

    def _generate_random_individual(self):
        """生成完全随机的新个体（用于多样性注入）"""
        indiv = np.zeros((self.params.d, self.params.r), dtype=int)
        remaining_S = self.params.S.copy()
        
        # 按随机顺序分配
        for j in np.random.permutation(self.params.d):
            for r in np.random.permutation(self.params.r):
                if remaining_S[j] <= 0: continue
                max_alloc = min(
                    remaining_S[j],
                    self.params.K[r] - np.sum(indiv[:, r])
                )
                indiv[j, r] = random.randint(0, max_alloc)
                remaining_S[j] -= indiv[j, r]
        
        return indiv

    def _crossover(self, p1, p2):
        """区域交换式交叉"""
        child = p1.copy()
        r = random.randint(0, self.params.r-1)  # 随机选择一个地区
        
        # 交换该地区的所有产品分配
        child[:, r] = p2[:, r]
        
        # 修复约束
        for j in range(self.params.d):
            if np.sum(child[j]) > self.params.S[j]:
                child[j] = self._repair_allocation(child[j], self.params.S[j])
        
        for r in range(self.params.r):
            if np.sum(child[:, r]) > self.params.K[r]:
                child[:, r] = self._repair_allocation(child[:, r], self.params.K[r])
        
        return child

    def _repair_allocation(self, alloc, max_total):
        """等比缩减超出部分"""
        if np.sum(alloc) <= max_total: return alloc
        scale = max_total / np.sum(alloc)
        return np.floor(alloc * scale).astype(int)

    def _mutate(self, indiv):
        """多维度强力变异"""
        new_indiv = indiv.copy()
        
        # 方案1：完全重置2个随机产品的分配（概率50%）
        if random.random() < 0.5:
            for j in random.sample(range(self.params.d), 2):
                # print(self.params.S[j])
                if self.params.S[j]<=0:
                    break
                new_indiv[j] = self._random_allocation_for_product(j, self.params.S[j], new_indiv)
        
        # 方案2：随机交换两个地区的分配（概率30%）
        elif random.random() < 0.3:
            r1, r2 = random.sample(range(self.params.r), 2)
            new_indiv[:, [r1, r2]] = new_indiv[:, [r2, r1]]  # 交换列
        
        return new_indiv

    def _random_allocation_for_product(self, j, total_S, current_indiv):
        """完全重新生成分配（确保不违反约束）"""
        alloc = np.zeros(self.params.r, dtype=int)
        
        # 计算剩余容量（确保非负）
        remaining_K = np.maximum(
            self.params.K - np.sum(current_indiv, axis=0) + current_indiv[j],
            0
        )
        
        # 按地区需求比例分配
        for r in np.argsort(-self.params.lambdas[j]):
            if total_S <= 0: 
                break
            max_possible = min(total_S, remaining_K[r])
            if max_possible <= 0:  # 跳过无效分配
                continue
            alloc[r] = random.randint(0, max_possible)
            total_S -= alloc[r]
            remaining_K[r] -= alloc[r]
        
        return alloc
    
    def _evaluate(self, indiv):
        """评估适应度（总期望销量）"""
        total = 0
        for j in range(self.params.d):
            for loc in range(self.params.r):
                z = indiv[j, loc]#获取当前方案对产品j和地区loc的分配值
                if z > 0:
                    z_vals, exp_vals = self.min_expec[(j, loc)]
                    total += np.interp(z, z_vals, exp_vals)

        
        return total
    
# 重复运行多次求平均
def run_multiple_times(params, n_runs=10, verbose=False):
    results = []
    solutions = []
    for i in range(n_runs):
        ga = SimpleGA(params)
        best_solution = ga.run()
        best_value = ga._evaluate(best_solution)
        results.append(best_value)
        solutions.append(best_solution)
        if verbose:
            print(f"Run {i+1}: {best_value:.2f}")
    avg_value = np.mean(results)
    std_value = np.std(results)
    best_idx = np.argmax(results)
    print("\n=== 多次运行统计 ===")
    print(f"运行次数: {n_runs}")
    print(f"平均期望销量: {avg_value:.2f}")
    print(f"标准差: {std_value:.2f}")
    print(f"最佳一次结果: {results[best_idx]:.2f}")
    return solutions[best_idx]  # 返回表现最好的解

if __name__ == "__main__":
    # 设置问题参数
    params = ProblemParameters(d=20, r=10)

    # 多次运行
    best_solution = run_multiple_times(params, n_runs=10, verbose=True)

    # 打印最好的解
    print("\n=== 最佳一次的解 ===")
    for j in range(params.d):
        row_str = ", ".join([f"地区{r}={best_solution[j,r]}" for r in range(params.r)])
        print(f"产品{j}: {row_str} (sum={np.sum(best_solution[j])}/{params.S[j]})")
    print(f"总期望销量: {SimpleGA(params)._evaluate(best_solution):.1f}")
    for r in range(params.r):
        print(f"地区{r}总分配: {np.sum(best_solution[:,r])}/{params.K[r]}")



17.0
29
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[4.139937718785167e-08, 7.037894121934784e-07, 5.982210003644566e-06, 3.389919002065254e-05, 0.0001440715575877733, 0.0004898432957984292, 0.0013878893380955494, 0.0033705883925177634, 0.007162500334100246, 0.013529167297744909, 0.022999584406166347, 0.035544812264075264, 0.050355150707439955, 0.0658490432328061, 0.07995955249697884, 0.09062082616324268, 0.09628462779844535, 0.09628462779844535, 0.09093548180964281, 0.08136332582968041, 0.06915882695522836, 0.05598571705899438, 0.04326169045467748, 0.03197603207519639, 0.022649689386597453, 0.015401788782886266, 0.010070400358041019, 0.006340622447655457, 0.0038496636289336696, 0.0022566993686852547]
正在初始化50个可行解...
初始化完成

=== 遗传算法开始 ===
+--------+----------+----------+----------+----------------------------------------+
|   代数 |   最佳值 |   平均值 |   最差值 | 最佳分配示例                           |
+========+==========+==========+